In [14]:
import ipywidgets as w

In [15]:
intro_box = w.VBox(children=(w.HTML(value='<font size="+2"><b>CARA</b> Covid Calculator</font>'),
                         w.Text(placeholder="E.g. Workshop without masks", description="Simulation name:",
                                style={'description_width': '35%'}),
                         w.Text(placeholder="E.g. 17/R-033", description="Room number:",
                                style={'description_width': '35%'})))

In [16]:
volume_text = w.Text(placeholder="Room volume (m³)", description="Room volume")
area_text = w.Text(placeholder="Room floor area (m²)", description="Floor area")
height_text = w.Text(placeholder="Room ceiling height (m²)", description="Ceiling height")

room_dimensions_box = w.VBox(children=(volume_text, w.Label(value="------- OR -------"),area_text, height_text),
                             layout=w.Layout(align_items='center'))
def manage_room_text_fields(_):
    volume_text.disabled = bool(area_text.value) or bool(height_text.value)
    area_text.disabled = bool(volume_text.value)
    height_text.disabled = bool(volume_text.value)

for text in [volume_text, area_text, height_text]:
    text.observe(manage_room_text_fields)
    text.style.description_width = "50%"

volume_text.observe(manage_room_text_fields)
area_text.observe(manage_room_text_fields)
height_text.observe(manage_room_text_fields)

In [145]:
ventilation_label = w.Label("Ventilation type:")
mechanical_button = w.ToggleButton(value=False, description='Mechanical', layout={'width': 'auto'})
natural_button = w.ToggleButton(value=False, description='Natural', layout={'width': 'auto'})

acph_text = w.Text(description="Air changes per hour", style={'description_width': 'auto'})
asfr_text = w.Text(description="Air supply flow rate", style={'description_width': 'auto'})

w.jsdlink((acph_text, 'value'), (asfr_text, 'disabled'))
w.jsdlink((asfr_text, 'value'), (acph_text, 'disabled'))

mech_vent_box = w.VBox(
    children=(acph_text, w.Label(value="------- OR -------"), asfr_text),
    layout=w.Layout(display='none')
)

nat_vent_box = w.VBox(
    children=(
        w.Text(description='Number of windows:', style={'description_width': 'auto'}),
        w.Text(description='Height of window:', placeholder='meters', style={'description_width': 'auto'}),
        w.Text(description='Width of window:', placeholder='meters', style={'description_width': 'auto'}),
        w.Text(description='Opening distance:', placeholder='centimeters', style={'description_width': 'auto'}),
        w.HBox(children=(
            w.Label('Windows open'),
            w.RadioButtons(options=('Always', '15 min / 2h'))
        ))
    ),
    layout=w.Layout(align_items='center', display='none'),
)



tab_bar = w.HBox(children=(mechanical_button, natural_button))
ventilation_type_menu = w.VBox(children=(ventilation_label, tab_bar), layout=w.Layout(align_items='center', width='auto'))
ventilation_menu = w.VBox(children=(ventilation_type_menu, mech_vent_box, nat_vent_box),
                          layout=w.Layout(align_items='center'))

def handle_natural_toggle(_):
    if natural_button.value:
        mechanical_button.value = False
        ventilation_menu.children[2].layout.display = 'block'
    else:
        ventilation_menu.children[2].layout.display = 'none'

def handle_mechanical_toggle(_):
    if mechanical_button.value:
        natural_button.value = False
        ventilation_menu.children[1].layout.display = 'block'
    else:
        ventilation_menu.children[1].layout.display = 'none'

natural_button.observe(handle_natural_toggle)
mechanical_button.observe(handle_mechanical_toggle)

ventilation_menu

In [ ]:
natural_button = w.ToggleButton(value=False, description='Mechanical ventilation', layout={'width': '100%'})
